In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from keras.layers import Activation
from keras.layers import Dropout
import pickle
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

2023-11-30 20:41:48.778225: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 20:41:48.778473: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 20:41:48.778731: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 20:41:48.847953: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov 30 20:41:52 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A5000                On | 00000000:00:07.0 Off |                  Off |
| 30%   31C    P8               22W / 230W|      1MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
cd trajectory_10s

/workspace/trajectory_10s


In [5]:
subjectList = pd.read_csv('participantID1.csv', header=None) # import the number participant IDs without the header
numSubjects = subjectList.count()[0]

subjectAlias = np.linspace(1,numSubjects,numSubjects).tolist()
outerSubjectAlias = subjectAlias
trialNames = ['NN','YN','NY','YY'] # NN = NNL + NNH, YN = YNL + YNH, NY = NYL + NHH, YY = YYL + YYH


In [6]:
with open('trajectoryDataFrame_win10.pkl', 'rb') as file:
    # Call load method to deserialze
    trajectoryDataFrame = pickle.load(file)
with open('outputDataFrame_win10.pkl', 'rb') as file:
    # Call load method to deserialze
    outputDataFrame = pickle.load(file)

In [7]:
# dropoutRate,LSTM1, LSTM2,numDenseLayers,features,seq_length, numClasses

parameterSpace = []

# Full parameter set
dropOutRates = [0.5,0.75]
firstLSTMNodes = [64, 128]
secondLSTMNodes = [64, 128]
numDenseLayers = [2,4,6]
miniBatch = [128,256]
numEpochs = [300]
featureAliases = [0, 1]
# featureCombo = [['xDot','yDot','zDot','thetaDot'], ['speed', 'thetaDot'], ['speed','thetaDot','theta']]
# thetaDot, theta, speed, accel, height
featureCombo = [['speed', 'thetaDot'], ['speed','thetaDot','height']]

# Parameter set 1
dropOutRates = [0.75]
firstLSTMNodes = [64, 128]
secondLSTMNodes = [64]
numDenseLayers = [2,4,6]
miniBatch = [256]
numEpochs = [300]  # make it 300
featureAliases = [0, 1]
# featureCombo = [['xDot','yDot','zDot','thetaDot'], ['speed', 'thetaDot'], ['speed','thetaDot','theta']]
# thetaDot, theta, speed, accel
# speed , thetaDot, height
# how long
featureCombo = [['speed', 'thetaDot'], ['speed','thetaDot','height']]

# Parameter set 2
dropOutRates = [0.5, 0.75]
firstLSTMNodes = [32, 64]
secondLSTMNodes = [64]
numDenseLayers = [2]
miniBatch = [256, 512]
numEpochs = [300]  # make it 300
featureAliases = [1]
# featureCombo = [['xDot','yDot','zDot','thetaDot'], ['speed', 'thetaDot'], ['speed','thetaDot','theta']]
# thetaDot, theta, speed, accel
# speed , thetaDot, height
# how long
featureCombo = [['speed', 'thetaDot'], ['speed','thetaDot','height']]

# Parameter set 3
dropOutRates = [0.25, 0.5]
firstLSTMNodes = [16, 32]
secondLSTMNodes = [32, 64]
numDenseLayers = [1, 2]
miniBatch = [256]
numEpochs = [300]  # make it 300
featureAliases = [1,2]
# featureCombo = [['xDot','yDot','zDot','thetaDot'], ['speed', 'thetaDot'], ['speed','thetaDot','theta']]
# thetaDot, theta, speed, accel
# speed , thetaDot, height
# how long
featureCombo = [['speed', 'thetaDot'], ['speed','thetaDot','height'],['height']]

# Parameter set 4
dropOutRates = [0.5]
firstLSTMNodes = [16]
secondLSTMNodes = [4, 16, 32]
numDenseLayers = [2]
miniBatch = [256]
numEpochs = [300]  # make it 300
featureAliases = [0, 1]
# featureCombo = [['xDot','yDot','zDot','thetaDot'], ['speed', 'thetaDot'], ['speed','thetaDot','theta']]
# thetaDot, theta, speed, accel
# speed , thetaDot, height
# how long
featureCombo = [['speed', 'thetaDot'], ['speed','thetaDot','height']]

# Parameter set 5
dropOutRates = [0.5, 0.75]
firstLSTMNodes = [16, 32, 64]
secondLSTMNodes = [4, 16, 32]
numDenseLayers = [2,4]
miniBatch = [256, 512]
numEpochs = [450]  # make it 300
featureAliases = [0, 1]
# featureCombo = [['xDot','yDot','zDot','thetaDot'], ['speed', 'thetaDot'], ['speed','thetaDot','theta']]
# thetaDot, theta, speed, accel
# speed , thetaDot, height
# how long
featureCombo = [['speed'], ['speed','thetaDot']]

for i in dropOutRates:
    for j in firstLSTMNodes:
        for k in secondLSTMNodes:
            for l in numDenseLayers:
                for m in miniBatch:
                    for n in numEpochs:
                        for o in featureAliases:
                            tempParamSpace = [i,j,k,l,m,n,o]
                            parameterSpace.append(tempParamSpace)

In [8]:
len(parameterSpace)

144

In [9]:
def assignUserOutput(dataFrameData, requiredSubjects):
    requiredData = dataFrameData.loc[requiredSubjects].to_numpy()
    return requiredData

In [9]:
def lstmModel(dropoutRate, LSTM1, LSTM2, numDenseLayers, features, seq_length, numClasses):
    init_learning_rate = 0.001
    model = Sequential()
    model.add(LSTM(LSTM1,input_shape=(seq_length,features),return_sequences=True)) # True = many to many
    model.add(LSTM(LSTM2,input_shape=(seq_length,features),return_sequences=False)) # To use fast cuda cores, use tanh activation functions
    for i in range(numDenseLayers) :
        model.add(Dropout(dropoutRate))
        model.add(Dense(16, activation='tanh'))
    model.add(Dense(numClasses,kernel_initializer='normal',activation='softmax'))
    #optimizer = tf.keras.optimizers.Adam(learning_rate=init_learning_rate, weight_decay=decay_rate)
    optimizer = tf.keras.optimizers.Adam(learning_rate=init_learning_rate)
    model.compile(loss='categorical_crossentropy',optimizer =optimizer,metrics=['accuracy'])
    return model

In [10]:
def assignUserTraj(dataFrameData, requiredSubjects, featureCombo, requiredFeatureCombo):
    requiredData = np.array([])
    for i in range(len(trajectoryDataFrame)):
        temp = dataFrameData[i].loc[requiredSubjects,featureCombo[requiredFeatureCombo]].to_numpy()
        temp = temp.reshape(temp.shape[0],1,temp.shape[1])
        if i == 0:
            requiredData = temp
        if i != 0 :
            requiredData = np.append(requiredData,temp, axis=1)
    return requiredData

In [38]:
print("Do you want to clear existing data? Type yes to clear and continue.")
a = input()

if a == "yes":
    print("Data Cleared.")
    i = 0
    j = 0
    k = 0
    outerParameterAccuracy = np.array([])
    outerParameterUsed = np.array([])
    innerParameterUsed = np.array([])
    innerParameterAccuracy = np.array([])

    innerParameterTrainAccuracy = np.array([])
    innerParameterTestAccuracy = np.array([])
    innerParameterUsedAggregate = np.array([])

    innerParameterTrainAccuracy = np.array([])
    innerParameterTestAccuracy = np.array([])
    innerParameterUsedAggregate = np.array([])

    innerAggregatedData = np.array([])
    outerAggregatedData = np.array([])
    with open('saveData/i_win10.pkl', 'wb') as file:
        pickle.dump(i, file)
    with open('saveData/j_win10.pkl', 'wb') as file:
        pickle.dump(j, file)
    with open('saveData/k_win10.pkl', 'wb') as file:
        pickle.dump(k, file)
    with open('saveData/outerParameterAccuracy_win10.pkl', 'wb') as file:
        pickle.dump(outerParameterAccuracy, file)
    with open('saveData/outerParameterUsed_win10.pkl', 'wb') as file:
        pickle.dump(outerParameterUsed, file)
    with open('saveData/innerParameterUsed_win10.pkl', 'wb') as file:
        pickle.dump(innerParameterUsed, file)
    with open('saveData/innerParameterAccuracy_win10.pkl', 'wb') as file:
        pickle.dump(innerParameterAccuracy, file)
    with open('saveData/innerAggregatedData_win10.pkl', 'wb') as file:
        pickle.dump(innerAggregatedData, file)
    with open('saveData/outerAggregatedData_win10.pkl', 'wb') as file:
        pickle.dump(outerAggregatedData, file)

Do you want to clear existing data? Type yes to clear and continue.


 yes


Data Cleared.


In [11]:
len(innerParameterUsed)

NameError: name 'innerParameterUsed' is not defined

In [12]:
K1 = 3
K2 = 3
trainFraction = 0.8 # defines fraction of training data
testFraction = 1 - trainFraction

In [1]:
with open('saveData/i_win10.pkl', 'rb') as file:
    i = pickle.load(file)
with open('saveData/j_win10.pkl', 'rb') as file:
    j= pickle.load(file)
with open('saveData/k_win10.pkl', 'rb') as file:
    k = pickle.load(file)
with open('saveData/outerParameterAccuracy_win10.pkl', 'rb') as file:
    outerParameterAccuracy = pickle.load(file)
with open('saveData/outerParameterUsed_win10.pkl', 'rb') as file:
    outerParameterUsed = pickle.load(file)
with open('saveData/innerParameterUsed_win10.pkl', 'rb') as file:
    innerParameterUsed = pickle.load(file)
with open('saveData/innerParameterAccuracy_win10.pkl', 'rb') as file:
    innerParameterAccuracy = pickle.load(file)
with open('saveData/innerAggregatedData_win10.pkl', 'rb') as file:
    innerAggregatedData = pickle.load(file)
with open('saveData/outerAggregatedData_win10.pkl', 'rb') as file:
    outerAggregatedData = pickle.load(file)

print('Outer Loop no. = {}, Inner Loop no. = {}, Parameter Space no. = {}'.format(i,j,k))
while i < K1:
    np.random.shuffle(outerSubjectAlias)
    numTrainDataOuter = int(trainFraction*len(outerSubjectAlias))
    numTestDataOuter = len(outerSubjectAlias) - numTrainDataOuter
    trainSubjectsOuterLoop = outerSubjectAlias[0:numTrainDataOuter]
    testSubjectsOuterLoop = outerSubjectAlias[len(trainSubjectsOuterLoop):len(outerSubjectAlias)]
    innerSubjectAlias = trainSubjectsOuterLoop


    while j < K2:
        print()
        print('Outer Loop no. = {}, Inner Loop no. = {}, Parameter Space no. = {}'.format(i,j,k))
        np.random.shuffle(innerSubjectAlias)
        numTrainDataInner = int(trainFraction*len(innerSubjectAlias))
        numTestDataInner = len(innerSubjectAlias) - numTrainDataInner
        trainSubjectsInnerLoop = outerSubjectAlias[0:numTrainDataInner]
        testSubjectsInnerLoop = outerSubjectAlias[len(trainSubjectsInnerLoop): len(trainSubjectsInnerLoop) + numTestDataInner]
        #for k, kk in zip(parameterSpace, range(len(parameterSpace))):
        while k < len(parameterSpace):
            requiredFeatureCombo = parameterSpace[k][6]
            trainTrajData = assignUserTraj(trajectoryDataFrame,trainSubjectsInnerLoop, featureCombo, requiredFeatureCombo)
            trainOutputData = assignUserOutput(outputDataFrame,trainSubjectsInnerLoop)

            testTrajData = assignUserTraj(trajectoryDataFrame,testSubjectsInnerLoop, featureCombo, requiredFeatureCombo)
            testOutputData = assignUserOutput(outputDataFrame,testSubjectsInnerLoop)

            # dropoutRate, LSTM1, LSTM2, numDenseLayers, features, seq_length, numClasses
            features = trainTrajData.shape[2]
            seq_length = trainTrajData.shape[1]
            numClasses = trainOutputData.shape[1]
            mlModel = lstmModel(parameterSpace[k][0], parameterSpace[k][1], parameterSpace[k][2], parameterSpace[k][3], features, seq_length, numClasses)
            # print("Parameter Tested: " + str((kk+1)/len(parameterSpace)))
            print("\r Inner Loop Progress: {:03.2f} %".format((k+1)/len(parameterSpace)), end='')
            innerHistory = mlModel.fit(trainTrajData,trainOutputData,epochs=parameterSpace[k][5],batch_size=parameterSpace[k][4],validation_data=(testTrajData, testOutputData),verbose=0)
            innerParameterAccuracy = np.append(innerParameterAccuracy, innerHistory.history['val_accuracy'][-1])
            #print("\n Parameter Set: {:02.1f},  Test Accuracy: {:03.3f} %".format((k+1)/len(parameterSpace),innerParameterAccuracy[-1]), end='')
            if  innerParameterUsed.shape[0]==0:
                innerParameterUsed = np.array(parameterSpace[k]).reshape(1,len(parameterSpace[k]))
            else:
                innerParameterUsed = np.append(innerParameterUsed, np.array(parameterSpace[k]).reshape(1,len(parameterSpace[k])), axis=0)

            if  innerAggregatedData.shape[0]==0:
                temp = [i, j,k,parameterSpace[k][0],parameterSpace[k][1],parameterSpace[k][2],parameterSpace[k][3],
                                                parameterSpace[k][4],parameterSpace[k][5],parameterSpace[k][6],
                                                innerHistory.history['val_accuracy'][-1],innerHistory.history['accuracy'][-1]]
                innerAggregatedData = np.array(temp).reshape(1,len(temp))
            else:
                temp = [i, j,k,parameterSpace[k][0],parameterSpace[k][1],parameterSpace[k][2],parameterSpace[k][3],
                                                parameterSpace[k][4],parameterSpace[k][5],parameterSpace[k][6],
                                                innerHistory.history['val_accuracy'][-1],innerHistory.history['accuracy'][-1]]
                innerAggregatedData = np.append(innerAggregatedData, np.array(temp).reshape(1,len(temp)), axis=0)

            k = k + 1
            with open('saveData/innerParameterUsed_win10.pkl', 'wb') as file:
                pickle.dump(innerParameterUsed, file)
            with open('saveData/innerParameterAccuracy_win10.pkl', 'wb') as file:
                pickle.dump(innerParameterAccuracy, file)
            with open('saveData/k_win10.pkl', 'wb') as file:
                pickle.dump(k, file)
            with open('saveData/innerAggregatedData_win10.pkl', 'wb') as file:
                pickle.dump(innerAggregatedData, file)
            with open("progress.txt", "a+") as file:
                file.write('Outer Loop no. = {}, Inner Loop no. = {}, Parameter Space no. = {}, Accuracy = {} \n'.format(i,j,k, innerHistory.history['val_accuracy'][-1]))
        j = j + 1
        k = 0
        with open('saveData/j_win10.pkl', 'wb') as file:
            pickle.dump(j, file)

    max_val = innerParameterAccuracy.max()
    maxAccuracyIndex = innerParameterAccuracy.argmax()
    optimalHyperparams = innerParameterUsed[maxAccuracyIndex] # Optimal Hyperparameters in the inner loop
    requiredFeatureCombo = int(optimalHyperparams[6])
    # Extract data for outer loop training
    trainTrajData = assignUserTraj(trajectoryDataFrame,trainSubjectsOuterLoop, featureCombo, requiredFeatureCombo)
    trainOutputData = assignUserOutput(outputDataFrame,trainSubjectsOuterLoop)

    testTrajData = assignUserTraj(trajectoryDataFrame,testSubjectsOuterLoop, featureCombo, requiredFeatureCombo)
    testOutputData = assignUserOutput(outputDataFrame,testSubjectsOuterLoop)

    # dropoutRate, LSTM1, LSTM2, numDenseLayers, features, seq_length, numClasses
    dropoutRate = int(optimalHyperparams[0])
    LSTM1_nodes = int(optimalHyperparams[1])
    LSTM2_nodes = int(optimalHyperparams[2])
    numDenseLayers = int(optimalHyperparams[3])
    numEpochs = int(optimalHyperparams[5])
    miniBatchSize = int(optimalHyperparams[4])
    requiredFeatureCombo = int(optimalHyperparams[6])
    features = trainTrajData.shape[2]
    seq_length = trainTrajData.shape[1]
    numClasses = trainOutputData.shape[1]

    # Train the data
    mlModel = lstmModel(dropoutRate, LSTM1_nodes, LSTM2_nodes, numDenseLayers, features, seq_length, numClasses)
    outerHistory = mlModel.fit(trainTrajData,trainOutputData,epochs=numEpochs,batch_size=miniBatchSize,validation_data=(testTrajData, testOutputData),verbose=0)
    outerParameterAccuracy = np.append(outerParameterAccuracy, outerHistory.history['val_accuracy'][-1])

    if  outerParameterUsed.shape[0]==0:
        outerParameterUsed = np.array(optimalHyperparams).reshape(1,len(optimalHyperparams))
    else:
        outerParameterUsed = np.append(outerParameterUsed, optimalHyperparams.reshape(1,len(optimalHyperparams)), axis=0)
    if  outerAggregatedData.shape[0]==0:
                temp = [i,optimalHyperparams[0],optimalHyperparams[1],optimalHyperparams[2],optimalHyperparams[3],
                                                optimalHyperparams[4],optimalHyperparams[5],optimalHyperparams[6],
                                                outerHistory.history['val_accuracy'][-1],outerHistory.history['accuracy'][-1]]
                outerAggregatedData = np.array(temp).reshape(1,len(temp))
    else:
        temp = [i,optimalHyperparams[0],optimalHyperparams[1],optimalHyperparams[2],optimalHyperparams[3],
                                                optimalHyperparams[4],optimalHyperparams[5],optimalHyperparams[6],
                                                outerHistory.history['val_accuracy'][-1],outerHistory.history['accuracy'][-1]]
        outerAggregatedData = np.append(outerAggregatedData, np.array(temp).reshape(1,len(temp)), axis=0)

    # save state
    i = i + 1
    j = 0
    innerParameterAccuracy = np.array([])
    innerParameterUsed = np.array([])
    print(" ")
    print("==================Saving Data========================")
    with open('saveData/i_win10.pkl', 'wb') as file:
        pickle.dump(i, file)
    with open('saveData/outerParameterAccuracy_win10.pkl', 'wb') as file:
        pickle.dump(outerParameterAccuracy, file)
    with open('saveData/outerParameterUsed_win10.pkl', 'wb') as file:
        pickle.dump(outerParameterUsed, file)
    with open('saveData/j_win10.pkl', 'wb') as file:
        pickle.dump(j, file)
    with open('saveData/k_win10.pkl', 'wb') as file:
        pickle.dump(k, file)
    with open('saveData/outerAggregatedData_win10.pkl', 'wb') as file:
        pickle.dump(outerAggregatedData, file)

FileNotFoundError: [Errno 2] No such file or directory: 'saveData/i_win10.pkl'